# 1. 상관분석

In [7]:
import pandas as pd
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
data = pd.DataFrame(diabetes['data'], columns=diabetes['feature_names'])

In [9]:
from scipy.stats import pearsonr
print(pearsonr(data['age'], data['bmi']))

PearsonRResult(statistic=0.1850846661465554, pvalue=9.07679186541741e-05)


In [11]:
print(data[['age', 'bmi']].corr())

          age       bmi
age  1.000000  0.185085
bmi  0.185085  1.000000


# 2. 정규성 검정

In [13]:
import numpy as np
np.random.seed(2024)
x = np.random.random(10)

In [15]:
from scipy.stats import shapiro
print(shapiro(x))

ShapiroResult(statistic=0.8969665765762329, pvalue=0.20285499095916748)


# 3. 모평균 검정